Nota generada a partir de [liga](https://www.dropbox.com/s/jwu8lu4r14pb7ut/3.2.1.Sistemas_de_ecuaciones_lineales_eliminacion_Gaussiana_y_factorizacion_LU.pdf?dl=0), [liga2](https://www.dropbox.com/s/s4ch0ww1687pl76/3.2.2.Factorizaciones_matriciales_SVD_Cholesky_QR.pdf?dl=0)

# 3.3.Solución de Sistemas de Ecuaciones Lineales y Factorizaciones Matriciales

En esta nota revisamos métodos por bloques para resolver sistemas de ecuaciones lineales:

## Sistemas de ecuaciones lineales (SEL)

En general son de la forma: $$\begin{array}{ccc} a_{11}x_1 + a_{12}x_2 + \cdots + a_{1n}x_n  &= & b_1 \\ a_{21}x_1 + a_{22}x_2 +  \cdots + a_{2n}x_n &= & b_2 \\ \vdots & & \\ a_{m1}x_1 + a_{m2}x_2 + \cdots + a_{mn}x_n &=& b_m \end{array}$$

donde: las $x_i$'s son las incógnitas y las $a_i$'s y $b_i$'s son constantes conocidas.

Las entradas $a_{ij}$'s son llamadas coeficientes del sistema y el conjunto de $b_i$'s se le llama lado derecho del sistema. Si todas las $b_i$'s son iguales a $0$ el sistema se le nombra **homogéneo**.

**3 posibilidades para solución del sistema anterior:**

* Una única solución: sólo existe uno y sólo un conjunto de valores de $x_i$'s que satisfacen todas las ecuaciones simultáneamente y el sistema se nombra **consistente** o **no singular**.

* Ninguna solución: no existe ningún conjunto de valores de $x_i$'s que satisfacen todas las ecuaciones simultáneamente (el conjunto solución es vacío) y el sistema se nombra **inconsistente** o singular.

* Infinitas soluciones: hay una infinidad de conjuntos (distintos) de valores de las $x_i$'s que satisfacen todas las ecuaciones simultáneamente. **obs:** si un sistema tiene más de una solución entonces tiene una infinidad de soluciones y el sistema se nombra **consistente** o **no singular**.



## Interpretación geométrica

Resolver un sistema de ecuaciones lineales equivale a encontrar la intersección entre rectas, planos o hiperplanos (2,3 o n dimensiones respectivamente). Por ejemplo para un caso de dos dimensiones se tiene:

<img src="https://dl.dropboxusercontent.com/s/xbokziv6zpwry3y/algebra_lineal_1.jpg?dl=0" heigth="700" width="700">


El inciso a) representa un sistema de ecuaciones lineales sin solución, el inciso b) infinitas soluciones (en el dibujo ligeramente se desplazó hacia abajo una de las rectas para mostrar ambas) y el inciso c) una única solución. 

**Nombres dados a los sistemas dependiendo de sus dimensiones:**

* Si $m > n$ el sistema $Ax=b$ se nombra *overdetermined*: hay más ecuaciones que incógnitas. En general no existe $x \in \mathbb{R}^n$ que satisfaga el sistema por lo que se busca resolver el problema de encontrar $x \in \mathbb{R}^n$ tal que $||Ax-b||$ sea mínima. Si $||\cdot||$ es la norma Euclidiana, el problema anterior se le llama **mínimos cuadrados**.

* Si $m < n$ el sistema $Ax=b$ se nombra *underdetermined*: hay menos ecuaciones que incógnitas. El sistema tiene una infinidad de soluciones ó ninguna solución. Lo anterior depende del $rank(A)$. Si $rank(A)=m$ entonces $Ax=b$ tiene infinitas soluciones\*

\*Para ver esto considérese el resultado del álgebra lineal [Rank-nullity theorem](https://en.wikipedia.org/wiki/Rank%E2%80%93nullity_theorem) que en este contexto se escribe como $n = rank(A) + nullity(A) = m + nullity(A)$ $\therefore nullity(A) = n-m$. Si $x_0 \in \mathbb{R}^n$ es una solución de $Ax=b$ entonces $x_0 + N(A)$ es el conjunto de soluciones de $Ax=b$, donde $x_0 + N(A) = \{x_0 + z | z \in N(A)\}$. 

## Métodos o algoritmos numéricos para SEL

Existen una gran cantidad de métodos para resolver SEL. Típicamente se elige el método de acuerdo a los coeficientes de la matriz del sistema que determinan propiedades de la misma y sus dimensiones.

Los hay **iterativos** y **directos o basados en factorizaciones matriciales (FM)**. Entre los directos o basados en FM se encuentran:


* Eliminación Gaussiana o factorización LU.
* Factorización de Cholesky.
* Factorización QR.
* Descomposición en valores singulares (DVS o SVD por siglas en inglés).

y como ejemplo de los iterativos están:

* Jacobi.
* Gauss-Seidel.
* Gradiente conjugado.

Ambos métodos: iterativos y directos o basados en FM encuentran **sistemas de ecuaciones equivalentes** a partir de operaciones básicas del álgebra lineal. Dos sistemas de ecuaciones lineales son equivalentes si tienen el mismo conjunto solución.

Los métodos o algoritmos iterativos anteriores no realizan factorizaciones matriciales. Aunque no explotan características de la matriz como simetría, utilizan menos memoria, convergen bajo condiciones simples de verificar y son sencillos de implementar.

Los métodos directos o basados en FM  utilizan más consumo de memoria, tienen el potencial de utilizar operaciones de nivel BLAS $3$, siempre encuentran solución si la matriz del sistema es no singular y en general son más complicados de implementar que los iterativos.


Nombremos a los métodos o algoritmos numéricos anteriores nuestros **Métodos o Algoritmos Básicos del Álgebra Lineal** para resolver SEL o FM **(MBAL/ABAL para SEL o FM)**. Cada uno puede utilizarse para resolver SEL, FM o para otros propósitos. Por ejemplo la factorización QR también se utiliza en el cálculo de eigenvalores y eigenvectores en el algoritmo QR, ver [QR algorithm](https://en.wikipedia.org/wiki/QR_algorithm).


Los MBAL/ABAL para SEL o FM sirven de apoyo para resolver SEL por **bloques**. 

## Métodos o algoritmos numéricos por bloques para SEL

Como se revisó en la nota [3.1.Vectorizacion_BLAS_y_el_uso_del_cache_eficientemente](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/III.computo_matricial/3.1.Vectorizacion_BLAS_y_el_uso_del_cache_eficientemente.ipynb) los *blocking algorithms* son ricos en operaciones de nivel 3 de BLAS, simplifican notación matemática y su escritura ayuda a pensar en algoritmos paralelizables. Una desventaja es la cantidad de memoria RAM que utilizan vs métodos que no trabajan por bloques.


### Eliminación por bloque

Este método consiste en eliminar un subconjunto de variables y resolver un sistema más pequeño. Si el sistema más pequeño asociado a la submatriz puede resolverse con alguno de los MBAL/ABAL para SEL o FM anteriores entonces este método puede tener más eficiencia que uno que no trabaja por bloques.


Considérese un sistema de la forma: $Ax=b$ escrito como:

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}
\end{array}
\right] \cdot 
\left[\begin{array}{c}
x_1\\
x_2
\end{array}
\right] =
\left[\begin{array}{c}
b_1\\
b_2
\end{array}
\right]
\end{array}
$$

con $x_1 \in \mathbb{R}^{n_1}, x_2 \in \mathbb{R}^{n_2}, A_{11} \in \mathbb{R}^{n_1 \times n_1}, A_{22} \in \mathbb{R}^{n_2 \times n_2}, b_1 \in \mathbb{R}^{n_1}, b_2 \in \mathbb{R}^{n_2}$.

El método de eliminación por bloques consiste en que si $A_{11}$ es invertible, entonces se puede eliminar $x_1$ de las ecuaciones como sigue:

* De la primer ecuación por bloques: $A_{11}x_1 + A_{12}x_2 = b_1$ se obtiene: $x_1 = A_{11}^{-1}(b_1-A_{12}x_2)$.

* Sustituyendo esta relación en la segunda ecuación por bloques obtenemos la ecuación reducida: $(A_{22}-A_{21}A_{11}^{-1}A_{12})x_2 = b_2 - A_{21}A_{11}^{-1}b_1$.

**Obs:** estas dos ecuaciones anteriores son equivalentes al sistema original $Ax=b$. La matriz $A_{22}-A_{21}A_{11}^{-1}A_{12}$ se le nombra **complemento de Schur** del bloque $A_{11}$ en $A$ (ver [Schur complement](https://en.wikipedia.org/wiki/Schur_complement)) esto es: $S = A_{22}-A_{21}A_{11}^{-1}A_{12}$. Se tiene la propiedad siguiente: $S$ es no singular si y sólo si $A$ es no singular.

Con lo anterior el método de eliminación por bloque consiste: 

**Algoritmo**

Sean $A$ y $A_{11}$ no singulares. 

1) Calcular $A_{11}^{-1}A_{12}$ y $A_{11}^{-1}b_1$ teniendo cuidado en **no** calcular la inversa sino un sistema de ecuaciones lineales.

2) Calcular el complemento de Schur del bloque $A_{11}$ en $A$: $S = A_{22}-A_{21}A_{11}^{-1}A_{12}$. Calcular $ \hat{b} = b_2-A_{21}A_{11}^{-1}b_1$.

3) Resolver $Sx_2 = \hat{b}$.

4) Resolver $A_{11}x_1 = b_1-A_{12}x_2$.



**Comentario:** En los pasos que se requieren resolver sistemas de ecuaciones lineales se utilizan los MBAL/ABAL para SEL o FM.

### Factor-Solve method

El método de factor solve consiste en expresar a la matriz del sistema $Ax = b$ como un producto de la forma: $A = A_1A_2 \cdots A_k$. Si $A$ es no singular entonces: $x = A^{-1}b = A_k^{-1}A_{k-1}^{-1} \cdots A_1^{-1}b$.

De esta forma, $x$ puede calcularse de "derecha a izquierda":

**Algoritmo**
$$
\begin{eqnarray}
z_1&:=A_1^{-1}b \nonumber \\
z_2&:=A_2^{-1}z_1 &= A_2^{-1}A_1^{-1}b \nonumber\\
\vdots \nonumber\\
z_{k-1}&:=A_{k-1}^{-1}z_{k-2} &= A_{k-1}^{-1} \cdots A_1^{-1}b \nonumber\\
x &:=A_k^{-1}z_{k-1} &= A_k^{-1} \cdot A_{k-1}^{-1} \cdots A_1^{-1}b \nonumber
\end{eqnarray}
$$


**Comentarios:** 

* El paso que consiste en factorizar a la matriz $A$ es el paso nombrado **factor** y el usado en resolver el sistema $A_iz_i = z_{i-1}$ es nombrado **solve**.

* Normalmente $A_i$ tiene una estructura aprovechable (por ejemplo triangular superior/inferior) para resolver el sistema $A_iz_i = z_{i-1}$. Por ejemplo en el caso de la factorización $LU$ de $A$ se cumple $PA = LU$ con $P$ matriz de permutación, $L$ triangular inferior con $1$'s en la diagonal y $U$ triangular superior. Entonces:

    * Paso 1: encontrar factores $P,L,U$ tales que $PA=LU$.
    * Paso 2: resolver $Ld=Pb$.
    * Paso 3: resolver $Ux=d$.

## Método de eliminación por bloques escrito como un *factor-solve method*:

El método de *block elimination* puede escribirse como un *factor-solve method*:

Paso $1$:

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & A_{12}\\
A_{21} & A_{22}
\end{array}
\right]=
\left[\begin{array}{cc}
A_{11} & 0 \\
A_{21} & S
\end{array}
\right]
\left[\begin{array}{cc}
I & A_{11}^{-1}A_{12} \\
0 & I
\end{array}
\right]
\end{array}
$$



Paso $2$. Resolver por el método de sustitución hacia delante por bloques:

$$\begin{array}{l}
\left[ \begin{array}{cc}
A_{11} & 0\\
A_{21} & S
\end{array}
\right] \cdot
\left[\begin{array}{c}
z_1 \\
z_2
\end{array}
\right] = 
\left[\begin{array}{c}
b_1\\
b_2
\end{array}
\right]
\end{array}
$$

Paso $3$. Resolver por el método de sustitución hacia atrás por bloques:

$$\begin{array}{l}
\left[ \begin{array}{cc}
I & A_{11}^{-1}A_{12}\\
0 & I
\end{array}
\right] \cdot
\left[\begin{array}{c}
x_1 \\
x_2
\end{array}
\right] = 
\left[\begin{array}{c}
z_1\\
z_2
\end{array}
\right]
\end{array}
$$

**Comentario:** En los pasos que se requieren resolver sistemas de ecuaciones lineales se utilizan los MBAL/ABAL para SEL o FM.

**Referencias:**

*  G. H. Golub, C. F. Van Loan, Matrix Computations, John Hopkins University Press, 2013.

* [3.1.Vectorizacion_BLAS_y_el_uso_del_cache_eficientemente](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/III.computo_matricial/3.1.Vectorizacion_BLAS_y_el_uso_del_cache_eficientemente.ipynb)

* Ver [1_ecuaciones_lineales](https://github.com/ITAM-DS/Propedeutico/blob/master/Python/clases/3_algebra_lineal/1_ecuaciones_lineales.ipynb) para una introducción a sistemas de ecuaciones lineales y factorizaciones matriciales con Python3.